In [ ]:
dummy_texts = [
    "This is a document about cats and their dietary needs.",
    "Dogs require different nutrition compared to cats.",
    "Understanding feline behavior is crucial for pet owners.",
    "Veterinary care for small animals includes various treatments.",
    "Common parasitic infections in pets and their prevention.",
    "Importance of vaccination schedules for puppies and kittens.",
    "Dental hygiene for pets: tips and procedures.",
    "Managing chronic diseases in older pets.",
    "First aid techniques for pet emergencies.",
    "Recognizing symptoms of common pet illnesses.",
    "The benefits of spaying and neutering pets.",
    "Nutrition for active and working dogs.",
    "Preventative care measures for feline health.",
    "Behavioral training tips for puppies and adult dogs.",
    "Dealing with anxiety and stress in pets.",
    "Grooming essentials for healthy pet coats.",
    "Common skin allergies and their treatments.",
    "Understanding pet vaccinations and their importance.",
    "Obesity in pets: causes, prevention, and management.",
    "Senior pet care: special considerations and dietary needs."
]

# Handling User Input: Analysis Image

In [1]:
query = "What's going on with my cat? What should I do?" 
image_path = "./cat.jpg"

import os
import base64
import ollama # Ensure ollama library is installed (pip install ollama)

# --- Configuration for the image ---
# IMPORTANT: Adjust this path if your cat.jpg is in a different location
image_model = "minicpm-v:8b" # Or "llava:7b" or another suitable vision model you have installed via Ollama

# --- 1. Generate a textual summary of the image using an LLM ---
print(f" ⏳ Processing image: {image_path}")

image_summary = "Could not generate image summary." # Default in case of error
if not os.path.exists(image_path):
    print(f"Error: Image file not found at {image_path}. Please check the path.")
else:
    try:
        # Read and encode image in base64
        with open(image_path, 'rb') as f:
            image_data = base64.b64encode(f.read()).decode('utf-8')

        # Updated prompt for detailed image summarization
        image_summarization_prompt = """From a feline veterinary stand point, provide a highly detailed and objective 
                description of the image. Focus on all observable elements, actions, 
                objects, subjects, their attributes (e.g., color, size, texture), 
                their spatial relationships, and any discernible context or implied scene. 
                Describe any text present in the image. This description must be exhaustive 
                and purely factual, capturing every significant visual detail to serve as a 
                comprehensive textual representation for further analysis by another AI model. 
                If the image is entirely irrelevant or contains no discernible subject, 
                state "No relevant visual information."."""

        # Send image to ollama for vision model processing
        response = ollama.chat(
            model=image_model,
            messages=[
                {
                    'role': 'user',
                    'content': image_summarization_prompt,
                    'images': [image_data]
                }
            ]
        )
        image_summary = response['message']['content']
        print("--- Generated Image Summary ---")
        print(image_summary)

    except Exception as e:
        print(f"Error processing image with Ollama: {e}")

# This 'image_summary' can now be used along with your user's text query
# for retrieval or further processing in your RAG pipeline.

 ⏳ Processing image: ./cat.jpg
--- Generated Image Summary ---
The image depicts an indoor scene with a focus on a cat that appears to be in distress or discomfort. The feline has a predominantly gray coat with patches of white and dark brown fur, particularly noticeable around the face, neck, chest, and paws. Notably, there is significant redness and what seems like irritated skin areas, especially visible on its chest and right side. This could suggest an underlying medical condition or recent injury.

The cat's eyes are wide open with a look that might indicate fear or discomfort. Its ears are in a neutral position but slightly tipped backward which can be interpreted as a sign of stress or anxiety in cats. The animal is standing, supported by four visible paws; the right front paw and tail appear particularly disheveled.

Surrounding the cat are various objects indicating it's likely within a domestic setting: there's an empty food bowl with what appears to be kibble on its side su

# Handling User Input: Refine Query

In [2]:
import uuid
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

# Define the LLM for query refinement (using the same model as your RAG chain if appropriate)
query_refinement_model = ChatOllama(model="llama3.2")

# Prompt for query refinement
query_refinement_prompt = ChatPromptTemplate.from_template(
    """You are an intelligent assistant. Your task is to rephrase and expand the given user query \
into a more detailed and context-rich query that can be used to retrieve relevant information \
from a veterinary knowledge base. Use the provided image description to add visual context \
and relevant keywords to the refined query. Focus on adding relevant keywords, clarifying intent, \
and anticipating related information that might be helpful. The output should be a single, refined query.

Original query: {original_query}
Image description: {image_summary}"""
)

# Create the query refinement chain
query_refinement_chain = (
    {
        "original_query": RunnablePassthrough(),
        "image_summary": RunnablePassthrough()
    }
    | query_refinement_prompt
    | query_refinement_model
    | StrOutputParser()
)# --- Demonstration of query refinement and then retrieval with scores ---


print(f"Original user query: {query}")
print(f"Image Summary: {image_summary}")

refined_query = query_refinement_chain.invoke(
    {"original_query": query, "image_summary": image_summary}
)
print(f"Refined query: {refined_query}")


/var/folders/1y/_wk9p9f500d8xnl6kjnnkbp40000gn/T/ipykernel_1790/3946929287.py:14: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  query_refinement_model = ChatOllama(model="llama3.2")


Original user query: What's going on with my cat? What should I do?
Image Summary: The image depicts an indoor scene with a focus on a cat that appears to be in distress or discomfort. The feline has a predominantly gray coat with patches of white and dark brown fur, particularly noticeable around the face, neck, chest, and paws. Notably, there is significant redness and what seems like irritated skin areas, especially visible on its chest and right side. This could suggest an underlying medical condition or recent injury.

The cat's eyes are wide open with a look that might indicate fear or discomfort. Its ears are in a neutral position but slightly tipped backward which can be interpreted as a sign of stress or anxiety in cats. The animal is standing, supported by four visible paws; the right front paw and tail appear particularly disheveled.

Surrounding the cat are various objects indicating it's likely within a domestic setting: there's an empty food bowl with what appears to be k